In [26]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import string
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
movies = pd.read_excel('movies_metadata.xlsx')
print(movies.shape)
movies.head()

(250, 12)


,Movie_id,Title,Link,Year,Runtime,Censorship,Ratings,Rating_count,Tags,Summary,Director,Actors
0,tt0111161,The Shawshank Redemption,/title/tt0111161/?ref_=chttp_t_1,1994,142,R,9.3,3.00,"['Epic', 'Prison Drama', 'Drama', 'Back to top']",A banker convicted of uxoricide forms a friend...,"['Frank Darabont', 'Frank Darabont', 'Frank Da...","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'..."
1,tt0068646,The Godfather,/title/tt0068646/?ref_=chttp_t_2,1972,175,R,9.2,2.10,"['Epic', 'Gangster', 'Tragedy', 'Crime', 'Dram...",The aging patriarch of an organized crime dyna...,"['Francis Ford Coppola', 'Francis Ford Coppola...","['Marlon Brando', 'Al Pacino', 'James Caan', '..."
2,tt0468569,The Dark Knight,/title/tt0468569/?ref_=chttp_t_3,2008,152,PG-13,9.0,2.90,"['Action Epic', 'Epic', 'Superhero', 'Action',...",When a menace known as the Joker wreaks havoc ...,"['Christopher Nolan', 'Christopher Nolan', 'Ch...","['Christian Bale', 'Heath Ledger', 'Aaron Eckh..."
3,tt0071562,The Godfather Part II,/title/tt0071562/?ref_=chttp_t_4,1974,202,R,9.0,1.40,"['Epic', 'Gangster', 'Tragedy', 'Crime', 'Dram...",The early life and career of Vito Corleone in ...,"['Francis Ford Coppola', 'Francis Ford Coppola...","['Al Pacino', 'Robert De Niro', 'Robert Duvall..."
4,tt0050083,12 Angry Men,/title/tt0050083/?ref_=chttp_t_5,1957,96,Approved,9.0,0.89,"['Legal Drama', 'Psychological Drama', 'Crime'...",The jury in a New York City murder trial is fr...,"['Sidney Lumet', 'Sidney Lumet', 'Sidney Lumet']","['Henry Fonda', 'Lee J. Cobb', 'Martin Balsam'..."


In [3]:
movies.isna().sum()

,0
Movie_id,0
Title,0
Link,0
Year,0
Runtime,0
Censorship,4
Ratings,0
Rating_count,0
Tags,0
Summary,0


In [4]:
movies.duplicated().sum()

0

In [5]:
movies['Censorship']=movies['Censorship'].fillna(movies['Censorship'].mode()[0])
movies.isna().sum()

,0
Movie_id,0
Title,0
Link,0
Year,0
Runtime,0
Censorship,0
Ratings,0
Rating_count,0
Tags,0
Summary,0


In [6]:
scaler = MinMaxScaler()
movies['Year_n'] = scaler.fit_transform(movies[['Year']])
movies['Runtime_n'] = scaler.fit_transform(movies[['Runtime']])
movies['Ratings_n'] = scaler.fit_transform(movies[['Ratings']])
movies['Rating_count_n'] = scaler.fit_transform(movies[['Rating_count']])


In [7]:
movies.head()

,Movie_id,Title,Link,Year,Runtime,Censorship,Ratings,Rating_count,Tags,Summary,Director,Actors,Year_n,Runtime_n,Ratings_n,Rating_count_n
0,tt0111161,The Shawshank Redemption,/title/tt0111161/?ref_=chttp_t_1,1994,142,R,9.3,3.00,"['Epic', 'Prison Drama', 'Drama', 'Back to top']",A banker convicted of uxoricide forms a friend...,"['Frank Darabont', 'Frank Darabont', 'Frank Da...","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'...",0.708738,0.502591,1.000000,1.000000
1,tt0068646,The Godfather,/title/tt0068646/?ref_=chttp_t_2,1972,175,R,9.2,2.10,"['Epic', 'Gangster', 'Tragedy', 'Crime', 'Dram...",The aging patriarch of an organized crime dyna...,"['Francis Ford Coppola', 'Francis Ford Coppola...","['Marlon Brando', 'Al Pacino', 'James Caan', '...",0.495146,0.673575,0.923077,0.696970
2,tt0468569,The Dark Knight,/title/tt0468569/?ref_=chttp_t_3,2008,152,PG-13,9.0,2.90,"['Action Epic', 'Epic', 'Superhero', 'Action',...",When a menace known as the Joker wreaks havoc ...,"['Christopher Nolan', 'Christopher Nolan', 'Ch...","['Christian Bale', 'Heath Ledger', 'Aaron Eckh...",0.844660,0.554404,0.769231,0.966330
3,tt0071562,The Godfather Part II,/title/tt0071562/?ref_=chttp_t_4,1974,202,R,9.0,1.40,"['Epic', 'Gangster', 'Tragedy', 'Crime', 'Dram...",The early life and career of Vito Corleone in ...,"['Francis Ford Coppola', 'Francis Ford Coppola...","['Al Pacino', 'Robert De Niro', 'Robert Duvall...",0.514563,0.813472,0.769231,0.461279
4,tt0050083,12 Angry Men,/title/tt0050083/?ref_=chttp_t_5,1957,96,Approved,9.0,0.89,"['Legal Drama', 'Psychological Drama', 'Crime'...",The jury in a New York City murder trial is fr...,"['Sidney Lumet', 'Sidney Lumet', 'Sidney Lumet']","['Henry Fonda', 'Lee J. Cobb', 'Martin Balsam'...",0.349515,0.264249,0.769231,0.289562


In [8]:
movies = movies.drop(columns=['Link'])
movies.head()

,Movie_id,Title,Year,Runtime,Censorship,Ratings,Rating_count,Tags,Summary,Director,Actors,Year_n,Runtime_n,Ratings_n,Rating_count_n
0,tt0111161,The Shawshank Redemption,1994,142,R,9.3,3.00,"['Epic', 'Prison Drama', 'Drama', 'Back to top']",A banker convicted of uxoricide forms a friend...,"['Frank Darabont', 'Frank Darabont', 'Frank Da...","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'...",0.708738,0.502591,1.000000,1.000000
1,tt0068646,The Godfather,1972,175,R,9.2,2.10,"['Epic', 'Gangster', 'Tragedy', 'Crime', 'Dram...",The aging patriarch of an organized crime dyna...,"['Francis Ford Coppola', 'Francis Ford Coppola...","['Marlon Brando', 'Al Pacino', 'James Caan', '...",0.495146,0.673575,0.923077,0.696970
2,tt0468569,The Dark Knight,2008,152,PG-13,9.0,2.90,"['Action Epic', 'Epic', 'Superhero', 'Action',...",When a menace known as the Joker wreaks havoc ...,"['Christopher Nolan', 'Christopher Nolan', 'Ch...","['Christian Bale', 'Heath Ledger', 'Aaron Eckh...",0.844660,0.554404,0.769231,0.966330
3,tt0071562,The Godfather Part II,1974,202,R,9.0,1.40,"['Epic', 'Gangster', 'Tragedy', 'Crime', 'Dram...",The early life and career of Vito Corleone in ...,"['Francis Ford Coppola', 'Francis Ford Coppola...","['Al Pacino', 'Robert De Niro', 'Robert Duvall...",0.514563,0.813472,0.769231,0.461279
4,tt0050083,12 Angry Men,1957,96,Approved,9.0,0.89,"['Legal Drama', 'Psychological Drama', 'Crime'...",The jury in a New York City murder trial is fr...,"['Sidney Lumet', 'Sidney Lumet', 'Sidney Lumet']","['Henry Fonda', 'Lee J. Cobb', 'Martin Balsam'...",0.349515,0.264249,0.769231,0.289562


In [9]:
movies['Censorship'].value_counts()

,count
Censorship,
R,105
PG,39
PG-13,35
Approved,23
Not Rated,23
G,18
Passed,5
NC-17,1
TV-MA,1


In [10]:
movies = pd.get_dummies(movies, columns=['Censorship'])
movies.head()

,Movie_id,Title,Year,Runtime,Ratings,Rating_count,Tags,Summary,Director,Actors,...,Rating_count_n,Censorship_Approved,Censorship_G,Censorship_NC-17,Censorship_Not Rated,Censorship_PG,Censorship_PG-13,Censorship_Passed,Censorship_R,Censorship_TV-MA
0,tt0111161,The Shawshank Redemption,1994,142,9.3,3.00,"['Epic', 'Prison Drama', 'Drama', 'Back to top']",A banker convicted of uxoricide forms a friend...,"['Frank Darabont', 'Frank Darabont', 'Frank Da...","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'...",...,1.000000,False,False,False,False,False,False,False,True,False
1,tt0068646,The Godfather,1972,175,9.2,2.10,"['Epic', 'Gangster', 'Tragedy', 'Crime', 'Dram...",The aging patriarch of an organized crime dyna...,"['Francis Ford Coppola', 'Francis Ford Coppola...","['Marlon Brando', 'Al Pacino', 'James Caan', '...",...,0.696970,False,False,False,False,False,False,False,True,False
2,tt0468569,The Dark Knight,2008,152,9.0,2.90,"['Action Epic', 'Epic', 'Superhero', 'Action',...",When a menace known as the Joker wreaks havoc ...,"['Christopher Nolan', 'Christopher Nolan', 'Ch...","['Christian Bale', 'Heath Ledger', 'Aaron Eckh...",...,0.966330,False,False,False,False,False,True,False,False,False
3,tt0071562,The Godfather Part II,1974,202,9.0,1.40,"['Epic', 'Gangster', 'Tragedy', 'Crime', 'Dram...",The early life and career of Vito Corleone in ...,"['Francis Ford Coppola', 'Francis Ford Coppola...","['Al Pacino', 'Robert De Niro', 'Robert Duvall...",...,0.461279,False,False,False,False,False,False,False,True,False
4,tt0050083,12 Angry Men,1957,96,9.0,0.89,"['Legal Drama', 'Psychological Drama', 'Crime'...",The jury in a New York City murder trial is fr...,"['Sidney Lumet', 'Sidney Lumet', 'Sidney Lumet']","['Henry Fonda', 'Lee J. Cobb', 'Martin Balsam'...",...,0.289562,True,False,False,False,False,False,False,False,False


In [11]:
movies.drop(columns=['Censorship_Passed', 'Censorship_TV-MA','Censorship_NC-17'])

,Movie_id,Title,Year,Runtime,Ratings,Rating_count,Tags,Summary,Director,Actors,Year_n,Runtime_n,Ratings_n,Rating_count_n,Censorship_Approved,Censorship_G,Censorship_Not Rated,Censorship_PG,Censorship_PG-13,Censorship_R
0,tt0111161,The Shawshank Redemption,1994,142,9.3,3.00,"['Epic', 'Prison Drama', 'Drama', 'Back to top']",A banker convicted of uxoricide forms a friend...,"['Frank Darabont', 'Frank Darabont', 'Frank Da...","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'...",0.708738,0.502591,1.000000,1.000000,False,False,False,False,False,True
1,tt0068646,The Godfather,1972,175,9.2,2.10,"['Epic', 'Gangster', 'Tragedy', 'Crime', 'Dram...",The aging patriarch of an organized crime dyna...,"['Francis Ford Coppola', 'Francis Ford Coppola...","['Marlon Brando', 'Al Pacino', 'James Caan', '...",0.495146,0.673575,0.923077,0.696970,False,False,False,False,False,True
2,tt0468569,The Dark Knight,2008,152,9.0,2.90,"['Action Epic', 'Epic', 'Superhero', 'Action',...",When a menace known as the Joker wreaks havoc ...,"['Christopher Nolan', 'Christopher Nolan', 'Ch...","['Christian Bale', 'Heath Ledger', 'Aaron Eckh...",0.844660,0.554404,0.769231,0.966330,False,False,False,False,True,False
3,tt0071562,The Godfather Part II,1974,202,9.0,1.40,"['Epic', 'Gangster', 'Tragedy', 'Crime', 'Dram...",The early life and career of Vito Corleone in ...,"['Francis Ford Coppola', 'Francis Ford Coppola...","['Al Pacino', 'Robert De Niro', 'Robert Duvall...",0.514563,0.813472,0.769231,0.461279,False,False,False,False,False,True
4,tt0050083,12 Angry Men,1957,96,9.0,0.89,"['Legal Drama', 'Psychological Drama', 'Crime'...",The jury in a New York City murder trial is fr...,"['Sidney Lumet', 'Sidney Lumet', 'Sidney Lumet']","['Henry Fonda', 'Lee J. Cobb', 'Martin Balsam'...",0.349515,0.264249,0.769231,0.289562,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,tt0032976,. Rebecca,1940,130,8.1,0.15,"['Dark Romance', 'Psychological Thriller', 'Dr...",A self-conscious woman juggles adjusting to he...,"['Alfred Hitchcock', 'Alfred Hitchcock', 'Alfr...","['Laurence Olivier', 'Joan Fontaine', 'George ...",0.184466,0.440415,0.076923,0.040404,True,False,False,False,False,False
246,tt1454029,. The Help,2011,146,8.1,0.50,"['Period Drama', 'Drama', 'Back to top']",An aspiring author during the civil rights mov...,"['Tate Taylor', 'Tate Taylor', 'Tate Taylor']","['Viola Davis', 'Emma Stone', 'Octavia Spencer...",0.873786,0.523316,0.076923,0.158249,False,False,False,False,True,False
247,tt0061512,. Cool Hand Luke,1967,127,8.1,0.19,"['Prison Drama', 'Tragedy', 'Crime', 'Drama', ...",A laid-back Southern man is sentenced to two y...,"['Stuart Rosenberg', 'Stuart Rosenberg', 'Stua...","['Paul Newman', 'George Kennedy', 'Strother Ma...",0.446602,0.424870,0.076923,0.053872,True,False,False,False,False,False
248,tt11032374,. Demon Slayer: Kimetsu no Yaiba - The Movie: ...,2020,117,8.2,0.08,"['Adult Animation', 'Anime', 'Dark Fantasy', '...",After his family was brutally murdered and his...,"['Haruo Sotozaki', 'Haruo Sotozaki', 'Haruo So...","['Natsuki Hanae', 'Akari Kitô', 'Yoshitsugu Ma...",0.961165,0.373057,0.153846,0.016835,False,False,False,False,False,False


In [12]:
str1=movies.Director[0].split(',')[1].strip(string.punctuation+' ')
print(str1)

Frank Darabont


In [13]:
movies['Title'] = movies['Title'].apply(lambda x: x.strip(string.punctuation+' '))

In [14]:
movies['Director_1'] = movies['Director'].apply(lambda x: x.split(',')[0].strip(string.punctuation+' '))
movies['Director_1'].value_counts()

,count
Director_1,
Christopher Nolan,8
Stanley Kubrick,7
Steven Spielberg,7
Martin Scorsese,7
Akira Kurosawa,6
...,...
Brian De Palma,1
Michel Gondry,1
Thomas Vinterberg,1


In [15]:
movies['Actors_1'] = movies['Actors'].apply(lambda x: ",".join([i.strip(string.punctuation+' ') for i in  x.split(',')]))
movies['Actors_1'].head()

,Actors_1
0,"Tim Robbins,Morgan Freeman,Bob Gunton,Tim Robb..."
1,"Marlon Brando,Al Pacino,James Caan,Marlon Bran..."
2,"Christian Bale,Heath Ledger,Aaron Eckhart,Chri..."
3,"Al Pacino,Robert De Niro,Robert Duvall,Al Paci..."
4,"Henry Fonda,Lee J. Cobb,Martin Balsam,Henry Fo..."


In [16]:
movies['Tags_1'] = movies['Tags'].apply(lambda x: " ".join([i.strip(string.punctuation+' ') for i in x.split(',')[:-1]]))
movies['Tags_1'].head()

,Tags_1
0,Epic Prison Drama Drama
1,Epic Gangster Tragedy Crime Drama
2,Action Epic Epic Superhero Action Crime Drama ...
3,Epic Gangster Tragedy Crime Drama
4,Legal Drama Psychological Drama Crime Drama


In [21]:
english_stopwords = stopwords.words('english')
movies['Summary_1']=movies['Summary'].apply(lambda x: ' '.join(i.lower() for i in word_tokenize(x) if i.lower() not in english_stopwords))

In [27]:
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

movies["Summary_l"] = movies["Summary_1"].apply(lemmatize_words)
movies["Summary_l"].head()

,Summary_l
0,banker convict uxoricide form friendship quart...
1,age patriarch organize crime dynasty transfer ...
2,menace know joker wreaks havoc chaos people go...
3,early life career vito corleone 1920s new york...
4,jury new york city murder trial frustrate sing...


In [28]:
movies['Summary_l'].head()

,Summary_l
0,banker convict uxoricide form friendship quart...
1,age patriarch organize crime dynasty transfer ...
2,menace know joker wreaks havoc chaos people go...
3,early life career vito corleone 1920s new york...
4,jury new york city murder trial frustrate sing...


In [29]:
tvec1 = TfidfVectorizer(ngram_range=(1, 2))

tvec1.fit(movies['Summary_l'])

#transform training data into sparse matrix
summary_tvec = tvec1.transform(movies['Summary_l'])

In [30]:
summary_tvec.shape

(250, 5836)

In [31]:
tvec2 = TfidfVectorizer()
tvec2.fit(movies['Tags_1'])

#transform training data into sparse matrix
tags_tvec = tvec2.transform(movies['Tags_1'])

In [32]:
tags_tvec.shape

(250, 140)

In [33]:
tvec3 = TfidfVectorizer(ngram_range=(2, 2))
actors_tvec = tvec3.fit_transform(movies['Actors_1'])
directors_tvec = tvec3.fit_transform(movies['Director_1'])


In [34]:
tvec3.vocabulary_

{'frank darabont': 48,
 'francis ford': 46,
 'ford coppola': 45,
 'christopher nolan': 20,
 'sidney lumet': 141,
 'peter jackson': 119,
 'steven spielberg': 146,
 'quentin tarantino': 122,
 'sergio leone': 139,
 'robert zemeckis': 131,
 'david fincher': 28,
 'irvin kershner': 67,
 'lana wachowski': 90,
 'martin scorsese': 98,
 'milos forman': 106,
 'frank capra': 47,
 'akira kurosawa': 2,
 'jonathan demme': 84,
 'fernando meirelles': 42,
 'roberto benigni': 132,
 'james cameron': 69,
 'george lucas': 51,
 'hayao miyazaki': 63,
 'roman polanski': 134,
 'bong joon': 10,
 'joon ho': 85,
 'alfred hitchcock': 4,
 'ridley scott': 126,
 'roger allers': 133,
 'joaquim dos': 74,
 'dos santos': 37,
 'damien chazelle': 24,
 'isao takahata': 68,
 'tony kaye': 158,
 'luc besson': 94,
 'masaki kobayashi': 99,
 'michael curtiz': 103,
 'bryan singer': 13,
 'olivier nakache': 113,
 'denis villeneuve': 36,
 'giuseppe tornatore': 56,
 'charles chaplin': 18,
 'andrew stanton': 5,
 'vidhu vinod': 161,
 'vi

In [35]:
print(directors_tvec)

  (0, 48)	1.0
  (1, 46)	0.7071067811865476
  (1, 45)	0.7071067811865476
  (2, 20)	1.0
  (3, 46)	0.7071067811865476
  (3, 45)	0.7071067811865476
  (4, 141)	1.0
  (5, 119)	1.0
  (6, 146)	1.0
  (7, 122)	1.0
  (8, 119)	1.0
  (9, 139)	1.0
  (10, 131)	1.0
  (11, 119)	1.0
  (12, 28)	1.0
  (13, 20)	1.0
  (14, 67)	1.0
  (15, 90)	1.0
  (16, 98)	1.0
  (17, 106)	1.0
  (18, 20)	1.0
  (19, 28)	1.0
  (20, 47)	1.0
  (21, 2)	1.0
  (22, 84)	1.0
  :	:
  (226, 128)	1.0
  (227, 167)	1.0
  (228, 166)	1.0
  (229, 135)	1.0
  (230, 11)	1.0
  (231, 160)	1.0
  (232, 141)	1.0
  (233, 127)	1.0
  (234, 91)	1.0
  (235, 130)	1.0
  (236, 170)	1.0
  (237, 40)	1.0
  (238, 115)	0.7071067811865476
  (238, 17)	0.7071067811865476
  (239, 138)	1.0
  (240, 55)	1.0
  (241, 11)	1.0
  (242, 78)	1.0
  (243, 61)	1.0
  (244, 3)	1.0
  (245, 4)	1.0
  (246, 149)	1.0
  (247, 147)	1.0
  (248, 62)	1.0
  (249, 148)	1.0


In [36]:
print(actors_tvec.shape)
print(directors_tvec.shape)

(250, 1365)
(250, 171)


In [37]:
combined_text_features = np.hstack([
    0.4 * summary_tvec.toarray(),      # Plot summaries given higher weight
    0.2 * actors_tvec.toarray(),    # Actors given moderate weight
    0.1 * directors_tvec.toarray(), # Directors given lesser weight
    0.3 * tags_tvec.toarray()     # Genres given moderate weight
])
combined_text_features.shape

(250, 7512)

In [38]:
text_feature_df = pd.DataFrame(combined_text_features)
text_feature_df.head()

,0,1,2,3,4,5,6,7,8,9,...,7502,7503,7504,7505,7506,7507,7508,7509,7510,7511
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.142865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.142865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
# Set the number of components to something smaller, like 100 or 200
svd = TruncatedSVD(n_components=100, random_state=42)
reduced_features = svd.fit_transform(text_feature_df)

In [40]:
reduced_features.shape

(250, 100)

In [41]:
similarity_matrix = cosine_similarity(reduced_features)

In [42]:
similarity_matrix.shape

(250, 250)

In [43]:
sorted_indices = np.argsort(similarity_matrix[127])[::-1]
print(sorted_indices)
for i in sorted_indices[0:10]:
    print(movies['Title'][i])

[127 182 229 207 175 179 169  59 212 194 189 185  72 234   2  79  21 142
 125 193 165 159 225  34 128 170  13  70  76  98 150  75 153 177 131 184
 114 129 121  43 248 108  16 113 107  41  85 221 173 219 124 235 152 102
  14 136 232 242  88 208 246 202   6 151 181 186 209 217 144 187  63 171
  51 241  27 139 216 116  92  44  48   0 222  31  26 215 197 180  96 213
 223 130 239  77 167 211  28 206 154  90  20 190 218 168 226 118 126 210
  83 224  57  62  40  33 106 145  38  18  81 195  17 220 109 237  58  66
  36   5 140 117 183  91  86 135 147 100 134   8 123  55  93  11 198  19
 120 203 111  37  71 205  49  29 137 214  69 146 133 115 161  54  39  82
  61 166  80 104   4   1  84 160 240  97 156 192 238   3 196  45  25 230
 143 103  12  47 112  60 138  24  10 201 204  78  22  94 162  89 149 243
 164 244 101 174 163  32  52  53 178  15   9 249 105 148  73  50 141 122
 172  67  99 231 191 199 110  30 188 233  42 119  74  64  87 155  65 176
 157 200  35 227 247  68 236 228  95 132  23  56   

In [44]:
movie = input(" Enter the movie name : ")
index_of_the_movie = movies[movies.Title == movie].index
print("Index of the movie:", index_of_the_movie)

# Getting a list of similar movies based on similarity scores
similarity_score = list(enumerate(similarity_matrix[index_of_the_movie]))
print("Similarity scores for similar movies:")
print(similarity_score)

# Sorting the movies based on their similarity score
sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)
print("Sorted similar movies based on similarity score:")
print(sorted_similar_movies)

 Enter the movie name : The Batman
Index of the movie: Index([], dtype='int64')
Similarity scores for similar movies:
[]
Sorted similar movies based on similarity score:
[]


In [46]:
# Getting Movie Name from User and finding recommendations
# Getting the movie name from the user
movie_name = input('Enter your favorite movie name: ')

# Finding the movie index
index_of_the_movie = movies[movies["Title"] == movie_name].index[0]
print("Index of the movie:", index_of_the_movie)
print("----------------------------------------")
sorted_indices = np.argsort(similarity_matrix[index_of_the_movie])[::-1]
i=0
for movie in sorted_indices[0:11]:
    if i==0:
      print("Your selected movie is: ",movies['Title'][movie])
      print("Other movies you might like: ")
    else:
      print(i,'-',movies['Title'][movie])
    i+=1

Enter your favorite movie name: Rocky
Index of the movie: 212
----------------------------------------
Your selected movie is:  Rocky
Other movies you might like: 
1 - Warrior
2 - Million Dollar Baby
3 - Raging Bull
4 - Children of Heaven
5 - Dangal
6 - Ford v Ferrari
7 - Rush
8 - The Grapes of Wrath
9 - Trainspotting
10 - Fight Club
